In [1]:
data_path = 'Breeds/'
model_weights = 'breed_classification.h5'

batch_size = 32
valid_size = 16
color_mode = 'rgb'

width = 224
height = 224

target_size = (width, height)
input_shape = (width, height, 3)

zoom_range = 0.3
shear_range = 0.3
shift_range = 0.3
rotation_range = 30

dense_1 = 512
dense_2 = 256
dense_3 = 64
num_classes = 5

epochs = 50
rate = 0.2

verbose = 1

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
############################################################################################

def preprocessing_function(img):
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img

def image_data_generator():
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rotation_range = rotation_range,
                                                                shear_range = shear_range,
                                                                zoom_range = zoom_range,
                                                                width_shift_range=shift_range,
                                                                height_shift_range=shift_range,
                                                                horizontal_flip = True,
                                                                preprocessing_function=preprocessing_function
                                                                )

    data_generator = train_datagen.flow_from_directory(
                                    data_path,
                                    target_size = target_size,
                                    color_mode = color_mode,
                                    batch_size = batch_size,
                                    class_mode = 'categorical',
                                    shuffle = True
                                    )

    return data_generator

In [3]:
data_generator = image_data_generator()
data_generator.class_indices

Found 1123 images belonging to 5 classes.


{'Ayrshire cattle': 0,
 'Brown Swiss cattle': 1,
 'Holstein Friesian cattle': 2,
 'Jersey cattle': 3,
 'Unknown': 4}

In [4]:
class CattleBreedClassification(object):
    def __init__(self):
        self.data_generator = data_generator
        self.train_step = self.data_generator.samples // batch_size

        self.accuracy = tf.keras.metrics.CategoricalAccuracy()
        self.recall = tf.keras.metrics.Recall()
        self.precision = tf.keras.metrics.Precision()

        self.id2breed = {v:k for k, v in self.data_generator.class_indices.items()}

    def classifier(self, x):
        if not self.trainable:
            x = Dense(dense_1)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

            x = Dense(dense_2)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

        x = Dense(dense_3)(x)
        x = BatchNormalization()(x)
        x = relu(x)
        x = Dropout(rate)(x)
        return x

    def model_conversion(self, trainable):
        functional_model = tf.keras.applications.MobileNetV2()
        functional_model.trainable = trainable

        self.trainable = trainable

        inputs = functional_model.input

        x = functional_model.layers[-2].output
        x = self.classifier(x)
        outputs = Dense(num_classes, activation='softmax')(x)

        model = Model(
                inputs=inputs,
                outputs=outputs
                    )
                    
        self.model = model
        self.model.summary()

    def train(self):
        callback = tf.keras.callbacks.EarlyStopping(
                                                monitor='loss', 
                                                patience=5
                                                    )

        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                          )
        self.model.fit(
                    self.data_generator,
                    steps_per_epoch= self.train_step,
                    epochs=epochs,
                    verbose=verbose
                        )

    def save_model(self):
        self.model.save(model_weights)

    def load_model(self):
        self.model = load_model(model_weights)
        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                         )

    def predict(self, x):
        x = cv.resize(x, target_size)
        x = preprocessing_function(x)
        x = np.expand_dims(x, axis=0)
        P = self.model.predict(x)
        idx = np.argmax(P)
        return self.id2breed[idx]
        
    def process(self):
        if not os.path.exists(model_weights):
            self.model_conversion(False)
            self.train()
            self.save_model()
        else:
            self.load_model()

In [5]:
model = CattleBreedClassification()
model.process()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

 tf.nn.relu_1 (TFOpLambda)      (None, 256)          0           ['batch_normalization_1[0][0]']  
                                                                                                  
 dropout_1 (Dropout)            (None, 256)          0           ['tf.nn.relu_1[0][0]']           
                                                                                                  
 dense_2 (Dense)                (None, 64)           16448       ['dropout_1[0][0]']              
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 64)          256         ['dense_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.nn.relu_2 (TFOpLambda)      (None, 64)           0           ['batch_normalization_2[0][0]']  
          

# **Inference**

In [6]:
img_path = 'Breeds/Holstein Friesian cattle/HolsteinFriesiancattle1.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Holstein Friesian cattle'

In [7]:
img_path = 'Breeds/Unknown/agalychnis-callidryas_39_c73eafa4.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Unknown'

In [8]:
img_path = 'Breeds/Brown Swiss cattle/BrownSwisscattle8.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Brown Swiss cattle'

In [12]:
img_path = 'Breeds/Ayrshire cattle/Ayrshirecattle147.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Ayrshire cattle'

In [11]:
img_path = 'Breeds/Jersey cattle/Jerseycattle99.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Jersey cattle'